# 1. Prepare Data
The 'extract_rgbnir_from_S2_folder' function creates a georeferenced .tiff of the RGB-NIR bands from the '.SAFE' data structure as provided by Scihub. The stacked image is saved in the same directory.

In [2]:
# if we have raw S2 download, create stacked image
from utils.rgbnir_stacked_from_S2_folder import extract_rgbnir_from_S2_folder
stack_needed=True
if stack_needed:
    folder_path = "/data2/simon/test_s2/S2A_MSIL2A_20230729T100031_N0509_R122_T33TUG_20230729T134559.SAFE"
    extract_rgbnir_from_S2_folder(folder_path)

writing band R ...
writing band G ...
writing band B ...
writing band NIR ...
Image succesfully written to: /data2/simon/test_s2/S2A_MSIL2A_20230729T100031_N0509_R122_T33TUG_20230729T134559.SAFE/stacked_RGBNIR.tif


# 2. SR functions
## 2.1 Create SR Object
This class holds all the information and function to SR a stacked RGB-NIR image. When initialized it holds the rasterio windowed reading coordinates with an overlap. The 'super_resolute_whole_image' function iteratively SRs the windows and cobines them together, averaging out the values for the overlapping areas.  
**Note some inefficiency problems:**
- Doesn't support multi-GPU inference
- The whole image and the initialized SR tensor are stored in memory
- images are SR'd one by one, not batched
- other functionalities, like histogram matching, run single-threadedly

In [3]:
# create object that holds window coordinates etc
from main import windowed_SR_and_saving
file_path="/data2/simon/test_s2/S2A_MSIL2A_20230729T100031_N0509_R122_T33TUG_20230729T134559.SAFE/stacked_RGBNIR.tif"
sr_obj = windowed_SR_and_saving(file_path)

## 2.2 Perform SR
The 'super_resolute_whole_image' takes the initialized model and iteratively performs the SR and the stitching & saving of the final image. Our models typically dont have a standard *model.forward* call to perform SR, which is why you can specify a custom command. Note that the normalization parameters are set in the class definitions.

In [ ]:
# SR and save whole image
sr_obj.super_resolute_whole_image(model=None,forward_call="forward",custom_steps=100)